In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import time

from func.propagation import prop2d, defmodel
from func.propagation_tf import simulate_obs
from func.inversion import gradiant

In [ ]:
# Import true model and observation
import pickle
filename = './data/v_ref'
infile = open(filename, 'rb')
v_ref = pickle.load(infile)
infile.close()

filename = './data/d_obs'
infile = open(filename, 'rb')
d_obs = pickle.load(infile)
infile.close()

# Initialize the model axis az, ax, at
nz,nx,nt = 28,28,235

vmin,vmax=2000,3000
az, ax, at, next, wsrc, zxsrc = defmodel(
                      vmin=vmin, vmax=vmax, fmax=25, # key parameters
                      nz=nz, nx=nx, nt=nt, # number of indices
                      izsrc=[nz//10], ixsrc=[nx//2],  # define source coordinates z, x
                      ext=10)  

# check_model(v=v_ref, vmin=vmin, vmax=vmax)
# Set receiver z and x coordinates
nrec = 10 # number of receivers per well
receivers_width = np.linspace(1,nx-1,num=nrec, endpoint=True, dtype=int)
zxrec = np.vstack((np.full(nrec, zxsrc[0]), receivers_width)) # Top
print("Number of receivers:", len(zxrec.flatten())//2)


v = tf.convert_to_tensor(np.zeros([nz, nx]) + 2500,dtype="float32")
with tf.GradientTape() as tape:
	tape.watch(v)
	start = time.time()
	d = simulate_obs(v, wsrc, zxsrc, at, az, ax, next, zxrec)
	print('Total time',time.strftime("%Hh %Mm %Ss", time.gmtime(time.time()-start)))
	# Calculate loss "J"
	loss = .5 * tf.reduce_sum((d - d_obs)**2)
	print("J = %.10f" % loss.numpy())
g = tape.gradient(loss, v)
print("gradient shape:", g.shape)

In [4]:
with tf.GradientTape() as tape:
	tape.watch(v)
	d = simulate_obs(v, wsrc, zxsrc, at, az, ax, next, zxrec)
	loss = .5 * tf.reduce_sum((d - d_obs)**2) #	Calculate loss parameter "J"
	print("J = %.10f" % loss.numpy())
g = tape.gradient(loss, v)

J = 77.9538269043


In [48]:
def prop2d_tf(vel,pwsrc,at,az,ax,next):
	nabs  = tf.constant(10)
	next2 = nabs + next
	nt    = tf.size(at)
	nz    = tf.size(az)
	nx    = tf.size(ax)
	az = tf.convert_to_tensor(az,dtype="float32")
	ax = tf.convert_to_tensor(ax,dtype="float32")
	at = tf.convert_to_tensor(at,dtype="float32")
	dz    = az[1] - az[0]
	dx    = ax[1] - ax[0]
	dt    = at[1] - at[0]   
	dz2   = 1./dz**2
	dx2   = 1./dx**2
	# Extend the model
	nze  = nz + 2*next2
	nxe  = nx + 2*next2
	########################################################
	#				    EXTEND MODEL				       #
	########################################################
	nze  = nz + 2*next2
	nxe  = nx + 2*next2
	vele = tf.zeros([nze,nxe],dtype="float32")
 
	# Central part
	vele = replace(vele,vel,[next2,nze-next2],[next2,nxe-next2])
 
	# Top and bottom
	vele = replace(vele,tf.repeat([vel[0,:]],next2,axis=0),
								 [0,next2],[next2,nxe-next2])
	vele = replace(vele,tf.repeat([vel[-1,:]],next2,axis=0),
	               [nze-next2,nze],[next2,nxe-next2]) 
	# Left and right
	vele = replace(vele,tf.transpose(tf.repeat([vel[:,0]],next2,axis=0)),
	                  [next2,nze-next2],[0,next2])
	vele = replace(vele,tf.transpose(tf.repeat([vel[:,-1]],next2,axis=0)),
	               		[next2,nze-next2],[nxe-next2,nxe])
	# Corners
	vele = replace(vele,tf.fill([next2, next2], vel[0,0]),
	               [0,next2],[0,next2])
	vele = replace(vele,tf.fill([next2, next2], vel[0,-1]),
	               [0,next2],[nxe-next2,nxe])
	vele = replace(vele,tf.fill([next2, next2], vel[-1,0]),
	               [nze-next2,nze],[0,next2])
	vele = replace(vele,tf.fill([next2, next2], vel[-1,-1]),
	               [nze-next2,nze],[nxe-next2,nxe])
	# vele = extend_model(vel,next2)
  ########################################################
	#
	########################################################
	# Shift the source by next
	p_all = tf.zeros([nze,nxe,2]) # All states wave field
	pm = tf.zeros([nze,nxe]) # Previous state wave field
	pp = tf.zeros([nze,nxe])# Current state wave field
	for it in tf.range(1,nt-1): # From 1 to nt-1
		start = time.time()
		fact = (dt*vele[1+nabs:-1-nabs,1+nabs:-1-nabs])**2
		lapx = (pp[1+nabs:-1-nabs,0+nabs:-2-nabs] - \
				2.*pp[1+nabs:-1-nabs,1+nabs:-1-nabs] + \
				pp[1+nabs:-1-nabs,2+nabs:-nabs])*dz2
		lapz = (pp[0+nabs:-2-nabs,1+nabs:-1-nabs] - \
				2.*pp[1+nabs:-1-nabs,1+nabs:-1-nabs] + \
				pp[2+nabs:-nabs,1+nabs:-1-nabs])*dx2
		asrc = tf.zeros([nze-2*nabs-2,nxe-2*nabs-2],dtype="float32")
		asrc = replace(asrc,pwsrc[1:-1,1:-1,it],
		              [next,nze-2*nabs-2-next],
               		[next,nxe-2*nabs-2-next])
					
		p_next = tf.zeros([nze,nxe])
		p_next = replace(p_next,
						2.*pp[1+nabs:-1-nabs,1+nabs:-1-nabs] - \
						pm[1+nabs:-1-nabs,1+nabs:-1-nabs] + \
						(lapz + lapx + asrc)*fact,
						[1+nabs,nze-1-nabs],[1+nabs,nxe-1-nabs])
		# One-way equation (bottom part)
		p_next = replace(p_next,
					pp[nze-1-nabs:nze,:nxe] - \
					vele[nze-1-nabs:nze,:nxe]*dt/dz* \
					(pp[nze-1-nabs:nze,:nxe]-pp[nze-2-nabs:nze-1,:nxe]),
					[nze-1-nabs,nze],[0,nxe])
		# One-way equation (top part)
		p_next = replace(p_next,
					pp[:1+nabs,:nxe] + \
					vele[:1+nabs,:nxe]*dt/dz* \
					(pp[1:2+nabs,:nxe]-pp[:1+nabs,:nxe]),
					[0,1+nabs],[0,nxe])
		# One-way equation (right part)
		p_next = replace(p_next,pp[:nze,nxe-1-nabs:nxe] - \
					vele[:nze,nxe-1-nabs:nxe]*dt/dx* \
					(pp[:nze,nxe-1-nabs:nxe] - pp[:nze,nxe-2-nabs:nxe-1]),
					[0,nze],[nxe-1-nabs,nxe])
		print(time.time()-start,'seconds for 1st part')
		start = time.time()
		# One-way equation (left part)
		p_next = replace(p_next,pp[:nze,:1+nabs] + \
					vele[:nze,:1+nabs]*dt/dx* \
					(pp[:nze,1:2+nabs]-pp[:nze,:1+nabs]),
					[0,nze],[0,1+nabs])
		p_all = tf.concat([p_all,tf.expand_dims(p_next,-1)],-1)
		# Update next state and save current state
		pm = pp
		pp = p_next
		print(time.time()-start,'seconds for 2nd part')
	return p_all[next2:nze-next2,next2:nxe-next2,:]

def simulate_obs(vel, wsrc, zxsrc, at, az, ax, next, zxrec):
	"""
	2d wave propagation with multiple sources
	"""
	nz,nx,nt = len(az),len(ax),len(at)
	vel = tf.convert_to_tensor(vel,dtype="float32")
	next = tf.constant(next)
	if wsrc.ndim == 1: wsrc=np.array([wsrc]) # convert source wavelet from 1d to 2d
	pwsrc = np.zeros([nz, nx, nt]) # creat a initial p with zeros
	pwsrc[zxsrc[0,:], zxsrc[1,:], :wsrc.shape[1]] = wsrc	# insert source wavelet
	pwsrc = tf.convert_to_tensor(pwsrc,dtype="float32")
	p = prop2d_tf(vel,pwsrc,at,az,ax,next) # propagate
	# Calculate observation at receivers' position
	nrec = zxrec.shape[1]
	d = tf.zeros([nrec,nt])
	for n in range(nrec):
		d = replace_1d(d,p[zxrec[0,n], zxrec[1,n], :],n,[0,nt])
	return d

In [6]:
def idx_to_replace(z1,z2,x1,x2):
  arr = []
  for i in range(z1,z2):
    for j in range(x1,x2):
      arr.append([i,j])
  return arr

def replace(tensor,arr,iz,ix):
	"""
	tensor: tensor to replace the elements
	arr: the array to insert
	iz,ix: the index of z and x
	"""
	replaced = tf.tensor_scatter_nd_update(
		tensor, 
    idx_to_replace(iz[0],iz[1],ix[0],ix[1]),
    tf.reshape(arr, [-1]))
	return replaced

def idx_to_replace_3d(z1,z2,x1,x2,t):
	arr = []
	for i in range(z1,z2):
		for j in range(x1,x2):
				arr.append([i,j,t]) 
	return arr

def replace_3d(tensor,arr,iz,ix,it):
	"""
	tensor: tensor to replace the elements
	arr: the array to insert
	iz,ix: the index of z and x
	"""
	return tf.tensor_scatter_nd_update(
		tensor, 
		idx_to_replace_3d(iz[0],iz[1],ix[0],ix[1],it),
		tf.reshape(arr, [-1]))

def idx_to_replace_1d(n,t1,t2):
  arr = []
  for t in range(t1,t2):
      arr.append([n,t])
  return arr

def replace_1d(tensor,arr,n,it):
	"""
	tensor: tensor to replace the elements
	arr: the array to insert
	iz,ix: the index of z and x
	"""
	replaced = tf.tensor_scatter_nd_update(
		tensor, 
    idx_to_replace_1d(n,it[0],it[1]),
    tf.reshape(arr, [-1]))
	return replaced